In [1]:
import pandas as pd
import numpy as np
from datetime import time
from numba import njit, prange
from itertools import product

In [2]:
# Load data
data_es = pd.read_csv('ES15Sec.txt', sep=',', header=0)
data_nq = pd.read_csv('NQ15Sec.txt', sep=',', header=0)

In [3]:
# display all columns
pd.set_option('display.max_columns', None)

In [4]:
# Check first 5 rows to understand the structure of the data
data_es.head()
# data_nq.head()

# check last 5 rows
# data_es.tail()
# data_nq.tail()


,Date,Time,Open,High,Low,Last,Volume,# of Trades,OHLC Avg,HLC Avg,HL Avg,Bid Volume,Ask Volume,fsvwap,dPOC,dVAH,dVAL,rthvwap,onPOC,onVAH,onVAL
0,2018-8-1,17:00:00.000000,3112.25,3112.25,3111.25,3112.00,649,247,3111.94,3111.83,3111.75,266,383,3111.8,0.0,0.0,0.0,0.0,3093.25,3100.75,3089.5
1,2018-8-1,17:00:15.000000,3112.00,3112.25,3111.75,3112.25,120,47,3112.06,3112.08,3112.00,51,69,3111.9,0.0,0.0,0.0,0.0,3093.25,3100.75,3089.5
2,2018-8-1,17:00:30.000000,3112.25,3112.25,3112.00,3112.25,62,30,3112.19,3112.17,3112.13,7,55,3111.9,0.0,0.0,0.0,0.0,3093.25,3100.75,3089.5
3,2018-8-1,17:00:45.000000,3112.25,3112.75,3112.00,3112.50,57,36,3112.38,3112.42,3112.38,15,42,3111.9,0.0,0.0,0.0,0.0,3093.25,3100.75,3089.5
4,2018-8-1,17:01:00.000000,3112.75,3112.75,3112.50,3112.75,14,10,3112.69,3112.67,3112.63,5,9,3111.9,0.0,0.0,0.0,0.0,3093.25,3100.75,3089.5


In [5]:
# check column names
data_es.columns
# data_nq.columns


Index(['Date', ' Time', ' Open', ' High', ' Low', ' Last', ' Volume',
       ' # of Trades', ' OHLC Avg', ' HLC Avg', ' HL Avg', ' Bid Volume',
       ' Ask Volume', ' fsvwap', ' dPOC', ' dVAH', ' dVAL', ' rthvwap',
       ' onPOC', ' onVAH', ' onVAL'],
      dtype='object')

In [6]:
# let's remove all whitespaces from the column names
data_es.columns = data_es.columns.str.replace(' ', '')

# same for NQ
data_nq.columns = data_nq.columns.str.replace(' ', '')

In [7]:
# Check for missing values
print(data_es.isna().sum())
# print(data_nq.isna().sum())

Date         0
Time         0
Open         0
High         0
Low          0
Last         0
Volume       0
#ofTrades    0
OHLCAvg      0
HLCAvg       0
HLAvg        0
BidVolume    0
AskVolume    0
fsvwap       0
dPOC         0
dVAH         0
dVAL         0
rthvwap      0
onPOC        0
onVAH        0
onVAL        0
dtype: int64


In [8]:
# Check for duplicate rows
print(data_es.duplicated().sum())
# print(data_nq.duplicated().sum())

0


In [9]:
# Check basic info 
print(data_es.info())
# print(data_nq.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8210745 entries, 0 to 8210744
Data columns (total 21 columns):
 #   Column     Dtype  
---  ------     -----  
 0   Date       object 
 1   Time       object 
 2   Open       float64
 3   High       float64
 4   Low        float64
 5   Last       float64
 6   Volume     int64  
 7   #ofTrades  int64  
 8   OHLCAvg    float64
 9   HLCAvg     float64
 10  HLAvg      float64
 11  BidVolume  int64  
 12  AskVolume  int64  
 13  fsvwap     float64
 14  dPOC       float64
 15  dVAH       float64
 16  dVAL       float64
 17  rthvwap    float64
 18  onPOC      float64
 19  onVAH      float64
 20  onVAL      float64
dtypes: float64(15), int64(4), object(2)
memory usage: 1.3+ GB
None


In [10]:
# Convert 'Date' and 'Time' columns to a single 'DateTime' column
data_es['DateTime'] = pd.to_datetime(data_es['Date'] + ' ' + data_es['Time'])
data_nq['DateTime'] = pd.to_datetime(data_nq['Date'] + ' ' + data_nq['Time'])

In [11]:
# drop 'Date' and 'Time' columns
data_es = data_es.drop(['Date', 'Time'], axis=1)
data_nq = data_nq.drop(['Date', 'Time'], axis=1)

In [12]:
# Set 'DateTime' as the index
data_es.set_index('DateTime', inplace=True)
data_nq.set_index('DateTime', inplace=True)


In [13]:
# sort the data by 'DateTime'
data_es.sort_index(inplace=True)
data_nq.sort_index(inplace=True)

In [14]:
# data_es.info()
data_es.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8210745 entries, 2018-08-01 17:00:00 to 2024-08-09 15:59:45
Data columns (total 19 columns):
 #   Column     Dtype  
---  ------     -----  
 0   Open       float64
 1   High       float64
 2   Low        float64
 3   Last       float64
 4   Volume     int64  
 5   #ofTrades  int64  
 6   OHLCAvg    float64
 7   HLCAvg     float64
 8   HLAvg      float64
 9   BidVolume  int64  
 10  AskVolume  int64  
 11  fsvwap     float64
 12  dPOC       float64
 13  dVAH       float64
 14  dVAL       float64
 15  rthvwap    float64
 16  onPOC      float64
 17  onVAH      float64
 18  onVAL      float64
dtypes: float64(15), int64(4)
memory usage: 1.2 GB


In [15]:
# data_es.head()
data_es.tail()

,Open,High,Low,Last,Volume,#ofTrades,OHLCAvg,HLCAvg,HLAvg,BidVolume,AskVolume,fsvwap,dPOC,dVAH,dVAL,rthvwap,onPOC,onVAH,onVAL
DateTime,,,,,,,,,,,,,,,,,,,
2024-08-09 15:58:45,5362.25,5362.75,5362.00,5362.75,122,71,5362.44,5362.50,5362.38,42,80,5353.6,5368.75,5373.75,5344.0,5354.98,0.0,0.0,0.0
2024-08-09 15:59:00,5362.75,5363.50,5362.50,5363.50,137,87,5363.06,5363.17,5363.00,25,112,5353.6,5368.75,5373.75,5344.0,5354.98,0.0,0.0,0.0
2024-08-09 15:59:15,5363.50,5364.00,5363.25,5364.00,116,60,5363.69,5363.75,5363.63,35,81,5353.6,5368.75,5373.75,5344.0,5354.98,0.0,0.0,0.0
2024-08-09 15:59:30,5364.00,5364.75,5363.75,5364.50,145,78,5364.25,5364.33,5364.25,83,62,5353.6,5368.75,5373.75,5344.0,5354.98,0.0,0.0,0.0
2024-08-09 15:59:45,5364.50,5365.00,5364.25,5365.00,84,74,5364.69,5364.75,5364.63,35,49,5353.6,5368.75,5373.75,5344.0,5354.98,0.0,0.0,0.0


In [16]:
data_nq.head()

,Open,High,Low,Last,Volume,#ofTrades,OHLCAvg,HLCAvg,HLAvg,BidVolume,AskVolume,fsvwap,dPOC,dVAH,dVAL,rthvwap,onPOC,onVAH,onVAL
DateTime,,,,,,,,,,,,,,,,,,,
2018-08-01 17:00:00,8818.50,8822.75,8818.50,8822.75,155,127,8820.63,8821.33,8820.63,72,83,8821.3,0.0,0.0,0.0,0.0,8757.25,8786.75,8750.0
2018-08-01 17:00:15,8823.25,8823.25,8822.00,8823.25,37,29,8822.94,8822.83,8822.63,25,12,8821.6,0.0,0.0,0.0,0.0,8757.25,8786.75,8750.0
2018-08-01 17:00:30,8823.25,8823.25,8822.00,8822.75,27,18,8822.81,8822.67,8822.63,13,14,8821.8,0.0,0.0,0.0,0.0,8757.25,8786.75,8750.0
2018-08-01 17:00:45,8823.00,8825.50,8822.50,8824.50,53,42,8823.88,8824.17,8824.00,7,46,8822.2,0.0,0.0,0.0,0.0,8757.25,8786.75,8750.0
2018-08-01 17:01:00,8825.25,8825.50,8824.25,8824.75,30,20,8824.94,8824.83,8824.88,19,11,8822.5,0.0,0.0,0.0,0.0,8757.25,8786.75,8750.0


In [17]:
# change "Last" column name to "Close"
data_es.rename(columns={'Last': 'Close'}, inplace=True)
data_nq.rename(columns={'Last': 'Close'}, inplace=True)

# change column names
data_es.rename(columns={'#ofTrades': 'NumOfTrades'}, inplace=True)
data_nq.rename(columns={'#ofTrades': 'NumOfTrades'}, inplace=True)

In [18]:
# Specify the columns to exclude from conversion
cols_to_exclude = ['BidVolume', 'AskVolume']

# Apply the conversion to data_es
data_es.loc[:, ~data_es.columns.isin(cols_to_exclude)] = data_es.loc[:, ~data_es.columns.isin(cols_to_exclude)].replace(0.0, np.nan)

# Apply the conversion to data_nq
data_nq.loc[:, ~data_nq.columns.isin(cols_to_exclude)] = data_nq.loc[:, ~data_nq.columns.isin(cols_to_exclude)].replace(0.0, np.nan)

In [19]:
# RTH start and end times
start_time = pd.to_datetime("08:30:00").time()
end_time = pd.to_datetime("16:00:00").time()

# Function to calculate dHi, dLo and dMid
def calculate_dHi_dLo_dMid(df):
    if not isinstance(df.index, pd.DatetimeIndex):
        raise TypeError("DataFrame index is not a DatetimeIndex.")
    
    # Add dHi, dLo, and dMid columns to df and fill with NaN
    df['dHi'] = np.nan
    df['dLo'] = np.nan
    df['dMid'] = np.nan
    
    # Date-based loop
    for date, group in df.groupby(df.index.date):
        day_mask = (group.index.time >= start_time) & (group.index.time <= end_time)
        day_data = group.loc[day_mask]
        
        if not day_data.empty:
            dHi = day_data['High'].cummax()
            dLo = day_data['Low'].cummin()
            dMid = (dHi + dLo) / 2
            
            df.loc[day_data.index, 'dHi'] = dHi
            df.loc[day_data.index, 'dLo'] = dLo
            df.loc[day_data.index, 'dMid'] = dMid
    
    return df

# Apply the function to data_es
data_es = calculate_dHi_dLo_dMid(data_es)

# Apply the function to data_nq
data_nq = calculate_dHi_dLo_dMid(data_nq)

In [20]:
# Get the unique dates from the DataFrame index
dates_es = data_es.index.normalize().unique()
dates_nq = data_nq.index.normalize().unique()

# Function to fill NaN values with the values from 08:29:45
def fill_with_early_values(df, dates):
    for date in dates:
        try:
            # Get the values from 08:29:45
            value_row = df.loc[date.strftime('%Y-%m-%d 08:29:45'), ['onPOC', 'onVAH', 'onVAL']]
            
            # Define the time range from 08:30:00 to 15:59:45
            date_start = date.strftime('%Y-%m-%d 08:30:00')
            date_end = date.strftime('%Y-%m-%d 16:00:00')
            mask = (df.index >= date_start) & (df.index <= date_end)
            
            # Fill NaN values within this time range with the values from 08:29:45
            for col in ['onPOC', 'onVAH', 'onVAL']:
                df.loc[mask, col] = df.loc[mask, col].fillna(value_row[col])
        except KeyError:
            continue
    return df

# Apply the function to both data_es and data_nq
data_es = fill_with_early_values(data_es, dates_es)
data_nq = fill_with_early_values(data_nq, dates_nq)

In [21]:
# IBH and IBL calculations 

@njit(parallel=True)
def calculate_high_low(df_values, time_values, start_hour, start_minute, end_hour, end_minute):
    n = len(df_values)
    high_values = np.full(n, np.nan)
    low_values = np.full(n, np.nan)
    
    # Calculate the start and end times in minutes
    start_time = start_hour * 60 + start_minute
    end_time = end_hour * 60 + end_minute

    current_high = -np.inf
    current_low = np.inf
    current_day = time_values[0] // (24 * 60 * 60 * 1e9)  # Determine the day

    for i in prange(n):
        day = time_values[i] // (24 * 60 * 60 * 1e9)
        minutes = (time_values[i] % (24 * 60 * 60 * 1e9)) // (60 * 1e9)

        if day != current_day:
            current_day = day
            current_high = -np.inf
            current_low = np.inf

        if start_time <= minutes < end_time:
            high = df_values[i][1]  # High column
            low = df_values[i][2]  # Low column
            if high > current_high:
                current_high = high
            if low < current_low:
                current_low = low
        elif minutes >= end_time and minutes < (16 * 60):
            high_values[i] = current_high
            low_values[i] = current_low

    return high_values, low_values

# Function to calculate IBH and IBL values for both data_es and data_nq
def calculate_ibh_ibl(df):
    df_values = df[['Open', 'High', 'Low', 'Close', 'Volume']].values
    time_values = df.index.view(np.int64)  # Convert datetime64 to int64
    
    # IBH5 and IBL5
    start_hour, start_minute = 8, 30
    end_hour, end_minute = 8, 35
    df['IBH5'], df['IBL5'] = calculate_high_low(df_values, time_values, start_hour, start_minute, end_hour, end_minute)

    # IBH15 and IBL15
    end_hour, end_minute = 8, 45
    df['IBH15'], df['IBL15'] = calculate_high_low(df_values, time_values, start_hour, start_minute, end_hour, end_minute)

    # IBH30 and IBL30
    end_hour, end_minute = 9, 0
    df['IBH30'], df['IBL30'] = calculate_high_low(df_values, time_values, start_hour, start_minute, end_hour, end_minute)

    # IBH and IBL
    end_hour, end_minute = 9, 30
    df['IBH'], df['IBL'] = calculate_high_low(df_values, time_values, start_hour, start_minute, end_hour, end_minute)

    # Set NaN values after 15:59:59
    df.loc[df.index.time > pd.Timestamp('15:59:59').time(), ['IBH5', 'IBL5', 'IBH15', 'IBL15', 'IBH30', 'IBL30', 'IBH', 'IBL']] = np.nan
    df.loc[df.index.time < pd.Timestamp('08:35:00').time(), ['IBH5', 'IBL5']] = np.nan
    df.loc[df.index.time < pd.Timestamp('08:45:00').time(), ['IBH15', 'IBL15']] = np.nan
    df.loc[df.index.time < pd.Timestamp('09:00:00').time(), ['IBH30', 'IBL30']] = np.nan
    df.loc[df.index.time < pd.Timestamp('09:30:00').time(), ['IBH', 'IBL']] = np.nan

    return df

# Apply the function to both data_es and data_nq
data_es = calculate_ibh_ibl(data_es)
data_nq = calculate_ibh_ibl(data_nq)




OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [23]:
# ON calculations

@njit
def calculate_on_values(df_values, time_values, start_hour, start_minute, end_hour, end_minute):
    n = len(df_values)
    on_high_values = np.full(n, np.nan)
    on_low_values = np.full(n, np.nan)
    on_mid_values = np.full(n, np.nan)
    
    current_high = -np.inf
    current_low = np.inf
    calculating_period = False

    start_seconds = start_hour * 3600 + start_minute * 60
    end_seconds = end_hour * 3600 + end_minute * 60
    
    for i in range(n):
        timestamp = time_values[i] / 1e9
        dt_seconds = timestamp % 86400  # Seconds in a day
        
        if dt_seconds == start_seconds:
            calculating_period = True
            current_high = -np.inf
            current_low = np.inf
        
        if calculating_period:
            high = df_values[i][1]  # High column
            low = df_values[i][2]  # Low column
            if high > current_high:
                current_high = high
            if low < current_low:
                current_low = low

        if dt_seconds >= end_seconds - 15 and dt_seconds < end_seconds + 45:  # Time period just before including the 08:30:00 candle
            calculating_period = False

            # Get the calculated value up to the candles just before including the 08:30:00 candle
            next_day_start_seconds = 8 * 3600 + 30 * 60
            next_day_end_seconds = 16 * 3600
            
            for j in range(i, n):
                next_timestamp = time_values[j] / 1e9
                next_dt_seconds = next_timestamp % 86400
                next_day_offset = int(next_timestamp // 86400) * 86400
                
                if next_day_offset + next_day_start_seconds <= next_timestamp < next_day_offset + next_day_end_seconds:
                    on_high_values[j] = current_high
                    on_low_values[j] = current_low
                    on_mid_values[j] = (current_high + current_low) / 2
                if next_timestamp >= next_day_offset + next_day_end_seconds:
                    break

    return on_high_values, on_low_values, on_mid_values

# Apply the function to both data_es and data_nq
for df in [data_es, data_nq]:
    df_values = df[['Open', 'High', 'Low', 'Close', 'Volume']].values
    time_values = df.index.view(np.int64)  # Convert datetime64 to int64 format
    
    # Calculate ONH, ONL, and ONMID
    start_hour, start_minute = 17, 0
    end_hour, end_minute = 8, 30
    df['ONH'], df['ONL'], df['ONMID'] = calculate_on_values(df_values, time_values, start_hour, start_minute, end_hour, end_minute)
    
    # Set ONH, ONL, and ONMID to NaN outside of 08:30 - 16:00 time range
    df.loc[(df.index.time < pd.Timestamp('08:30:00').time()) | (df.index.time >= pd.Timestamp('17:00:00').time()), ['ONH', 'ONL', 'ONMID']] = np.nan


In [24]:
# Function to find the previous trading day with data
def find_previous_data_day(df, reference_date):
    reference_date = pd.Timestamp(reference_date)
    current_date = reference_date - pd.Timedelta(days=1)  # Start from the previous day
    while current_date >= df.index.min():
        if current_date in df.index and not df.loc[str(current_date)].isnull().all():
            return current_date  # Return the first previous day with data
        current_date -= pd.Timedelta(days=1)
    return None  # Return None if no previous day with data is found

# Numba-optimized function to calculate RTH values (High, Low, Mid, Close)
@njit
def calculate_rth_values_numba(df_values, start_seconds, end_seconds):
    current_high = -np.inf
    current_low = np.inf
    current_close = np.nan
    
    for i in range(df_values.shape[0]):
        timestamp, high, low, close = df_values[i]
        if start_seconds <= timestamp % 86400 <= end_seconds:
            if high > current_high:
                current_high = high
            if low < current_low:
                current_low = low
            current_close = close
    
    if current_high > -np.inf and current_low < np.inf:
        current_mid = (current_high + current_low) / 2
    else:
        current_high = current_low = current_mid = current_close = np.nan
    
    return current_high, current_low, current_mid, current_close

# Function to calculate and update RTH values for the entire DataFrame
def calculate_and_update_rth_values(df):
    date_range = df.index.normalize().unique()

    for current_date in date_range:
        previous_data_day = find_previous_data_day(df, current_date)
        if previous_data_day:
            # Filter data from the previous day
            prev_day_data = df.loc[str(previous_data_day) + ' 08:30:00':str(previous_data_day) + ' 16:00:00']
            if not prev_day_data.empty:
                prev_day_values = prev_day_data[['High', 'Low', 'Close']].values
                prev_day_timestamps = prev_day_data.index.view(np.int64) // 1e9  # Convert datetime64 to int64 and then to seconds
                prev_day_df_values = np.column_stack((prev_day_timestamps, prev_day_values))

                # Calculate RTH values (High, Low, Mid, Close)
                start_seconds = 8 * 3600 + 30 * 60
                end_seconds = 16 * 3600
                pHi, pLo, pMid, pCl = calculate_rth_values_numba(prev_day_df_values, start_seconds, end_seconds)

                # Assign the calculated RTH values to the current date
                df.loc[str(current_date) + ' 08:30:00':str(current_date) + ' 16:00:00', 'pHi'] = pHi
                df.loc[str(current_date) + ' 08:30:00':str(current_date) + ' 16:00:00', 'pLo'] = pLo
                df.loc[str(current_date) + ' 08:30:00':str(current_date) + ' 16:00:00', 'pMid'] = pMid
                df.loc[str(current_date) + ' 08:30:00':str(current_date) + ' 16:00:00', 'pCl'] = pCl
            else:
                # If no data is available for the previous day, set current day values to NaN
                df.loc[str(current_date) + ' 08:30:00':str(current_date) + ' 16:00:00', ['pHi', 'pLo', 'pMid', 'pCl']] = np.nan
        else:
            # If no previous day with data is found, set current day values to NaN
            df.loc[str(current_date) + ' 08:30:00':str(current_date) + ' 16:00:00', ['pHi', 'pLo', 'pMid', 'pCl']] = np.nan

# Apply the function to both data_es and data_nq
for df in [data_es, data_nq]:
    calculate_and_update_rth_values(df)


In [25]:
# Numba-optimized function to calculate the score
# level based trend score calculation

@njit(parallel=True)
def calculate_score_numba(high, low, fsvwap, dPOC, dVAH, dVAL, rthvwap, onPOC, onVAH, onVAL, IBH, IBL, ONH, ONL, dMid, pHi, pLo, pMid, pCl):
    score = np.zeros(high.shape)
    
    for i in prange(high.shape[0]):
        if not np.isnan(fsvwap[i]):
            if high[i] > fsvwap[i]:
                score[i] += 5
            if low[i] < fsvwap[i]:
                score[i] -= 5

        if not np.isnan(dPOC[i]):
            if high[i] > dPOC[i]:
                score[i] += 5
            if low[i] < dPOC[i]:
                score[i] -= 5

        if not np.isnan(dVAH[i]):
            if high[i] > dVAH[i]:
                score[i] += 5
            if low[i] < dVAH[i]:
                score[i] -= 5

        if not np.isnan(dVAL[i]):
            if high[i] > dVAL[i]:
                score[i] += 5
            if low[i] < dVAL[i]:
                score[i] -= 5

        if not np.isnan(rthvwap[i]):
            if high[i] > rthvwap[i]:
                score[i] += 5
            if low[i] < rthvwap[i]:
                score[i] -= 5

        if not np.isnan(onPOC[i]):
            if high[i] > onPOC[i]:
                score[i] += 5
            if low[i] < onPOC[i]:
                score[i] -= 5

        if not np.isnan(onVAH[i]):
            if high[i] > onVAH[i]:
                score[i] += 5
            if low[i] < onVAH[i]:
                score[i] -= 5

        if not np.isnan(onVAL[i]):
            if high[i] > onVAL[i]:
                score[i] += 5
            if low[i] < onVAL[i]:
                score[i] -= 5

        if not np.isnan(pHi[i]):
            if high[i] > pHi[i]:
                score[i] += 5
            if low[i] < pHi[i]:
                score[i] -= 5

        if not np.isnan(pLo[i]):
            if high[i] > pLo[i]:
                score[i] += 5
            if low[i] < pLo[i]:
                score[i] -= 5

        if not np.isnan(pMid[i]):
            if high[i] > pMid[i]:
                score[i] += 5
            if low[i] < pMid[i]:
                score[i] -= 5

        if not np.isnan(pCl[i]):
            if high[i] > pCl[i]:
                score[i] += 5
            if low[i] < pCl[i]:
                score[i] -= 5

        if not np.isnan(IBH[i]):
            if high[i] > IBH[i]:
                score[i] += 15
        if not np.isnan(IBL[i]):
            if low[i] < IBL[i]:
                score[i] -= 15

        if not np.isnan(ONH[i]):
            if high[i] > ONH[i]:
                score[i] += 15
        if not np.isnan(ONL[i]):
            if low[i] < ONL[i]:
                score[i] -= 15

        if not np.isnan(dMid[i]):
            if high[i] > dMid[i]:
                score[i] += 10
            if low[i] < dMid[i]:
                score[i] -= 10

    return score

# Apply the function to both data_es and data_nq DataFrames
for df in [data_es, data_nq]:
    # Filter the DataFrame for trading hours (08:30 - 16:00)
    df_filtered = df.between_time('08:30:00', '16:00:00')
    
    # Calculate the score using the Numba function
    scores = calculate_score_numba(
        df_filtered['High'].values,
        df_filtered['Low'].values,
        df_filtered['fsvwap'].values,
        df_filtered['dPOC'].values,
        df_filtered['dVAH'].values,
        df_filtered['dVAL'].values,
        df_filtered['rthvwap'].values,
        df_filtered['onPOC'].values,
        df_filtered['onVAH'].values,
        df_filtered['onVAL'].values,
        df_filtered['IBH'].values,
        df_filtered['IBL'].values,
        df_filtered['ONH'].values,
        df_filtered['ONL'].values,
        df_filtered['dMid'].values,
        df_filtered['pHi'].values,
        df_filtered['pLo'].values,
        df_filtered['pMid'].values,
        df_filtered['pCl'].values
    )
    
    # Add the scores back to the original DataFrame
    df['score'] = np.nan
    df.loc[df_filtered.index, 'score'] = scores


In [26]:
# Create a new column for AvgTradeSize and round it to 2 decimal places
data_es['AvgTradeSize'] = (data_es['Volume'] / data_es['NumOfTrades']).round(2)
data_nq['AvgTradeSize'] = (data_nq['Volume'] / data_nq['NumOfTrades']).round(2)


In [27]:
# Function to detect half-days
def detect_half_days(df):
    # Identify days that have data at 08:30
    days_with_morning_data = df.between_time('08:30', '08:30').index.normalize()
    
    # Identify days that have data at 15:00
    days_with_afternoon_data = df.between_time('15:00', '15:00').index.normalize()
    
    # Half-days: days with morning data but no data at 15:00
    half_days = days_with_morning_data.difference(days_with_afternoon_data)
    
    return half_days

# Function to remove RTH data for identified half-days
def remove_half_day_data(df, half_days):
    # Remove data between 08:30 - 16:00 for half-days
    for day in half_days:
        df.drop(df.loc[day.strftime('%Y-%m-%d 08:30:00'):day.strftime('%Y-%m-%d 16:00:00')].index, inplace=True)
    
    return df

# Detect half-days
half_days_es = detect_half_days(data_es)
half_days_nq = detect_half_days(data_nq)

# Remove data for half-days
data_es_cleaned = remove_half_day_data(data_es, half_days_es)
data_nq_cleaned = remove_half_day_data(data_nq, half_days_nq)

data_es = data_es_cleaned
data_nq = data_nq_cleaned


In [28]:
# Function to fill missing rows only for valid trading days and hours
def fill_missing_data_correctly(df):
    # Identify valid trading days by checking if there is data at 08:30:00
    valid_days = df[df.index.time == pd.to_datetime('08:30:00').time()].index.normalize()

    # Create a complete time series (trading hours: 08:30 - 15:59:45) for these valid days
    full_time_index = pd.date_range(
        start=valid_days.min(), 
        end=valid_days.max(), 
        freq='15s'
    )
    
    # Filter the full time index to include only valid days
    full_time_index = full_time_index[full_time_index.normalize().isin(valid_days)]
    
    # Select only the trading hours
    full_time_index = full_time_index[(full_time_index.time >= pd.to_datetime('08:30:00').time()) & 
                                      (full_time_index.time <= pd.to_datetime('15:59:45').time())]
    
    # Reindex the original DataFrame with this complete time series
    df_reindexed = df.reindex(full_time_index)
    
    # Fill missing rows only within the trading hours for valid days
    df_filled = df_reindexed.ffill()
    
    # Holidays or non-trading days remain unfilled
    return df_filled

# Apply the function to data_es and data_nq
data_es_filled = fill_missing_data_correctly(data_es)
data_nq_filled = fill_missing_data_correctly(data_nq)

# Assign the filled DataFrame back to the original variables
data_es = data_es_filled
data_nq = data_nq_filled

In [29]:
# Function to create a complete time series (08:30:00 - 15:59:45, at 15-second intervals)
def create_full_time_range(date):
    return pd.date_range(start=f'{date} 08:30:00', end=f'{date} 15:59:45', freq='15s')

# Function to check the entire data for missing time intervals
def check_missing_times(df):
    missing_times_dict = {}
    
    # Iterate over all unique days in the dataset
    for day in df.index.normalize().unique():
        # If there is data at 08:30:00 on the given day
        if pd.Timestamp(f'{day} 08:30:00') in df.index:
            # Get the data series for that day
            daily_data = df[df.index.normalize() == day]
            
            # Create the complete time series for that day
            full_time_range = create_full_time_range(day.date())
            
            # Find the missing time intervals
            missing_times = full_time_range.difference(daily_data.index)
            
            # If there are missing intervals, store the day and missing times
            if not missing_times.empty:
                missing_times_dict[day] = missing_times
    
    return missing_times_dict

# Check for missing time intervals
missing_times_in_data = check_missing_times(data_es)
# missing_times_in_data = check_missing_times(data_nq)

# Print the results
if not missing_times_in_data:
    print("All data is complete and correctly filled.")
else:
    for day, missing_times in missing_times_in_data.items():
        print(f"Missing times on {day.date()}:")
        print(missing_times)

All data is complete and correctly filled.


In [30]:
data_es.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2703600 entries, 2018-08-02 08:30:00 to 2024-08-08 15:59:45
Data columns (total 39 columns):
 #   Column        Dtype  
---  ------        -----  
 0   Open          float64
 1   High          float64
 2   Low           float64
 3   Close         float64
 4   Volume        float64
 5   NumOfTrades   float64
 6   OHLCAvg       float64
 7   HLCAvg        float64
 8   HLAvg         float64
 9   BidVolume     float64
 10  AskVolume     float64
 11  fsvwap        float64
 12  dPOC          float64
 13  dVAH          float64
 14  dVAL          float64
 15  rthvwap       float64
 16  onPOC         float64
 17  onVAH         float64
 18  onVAL         float64
 19  dHi           float64
 20  dLo           float64
 21  dMid          float64
 22  IBH5          float64
 23  IBL5          float64
 24  IBH15         float64
 25  IBL15         float64
 26  IBH30         float64
 27  IBL30         float64
 28  IBH           float64
 29  IBL         

In [31]:
data_nq.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2703600 entries, 2018-08-02 08:30:00 to 2024-08-08 15:59:45
Data columns (total 39 columns):
 #   Column        Dtype  
---  ------        -----  
 0   Open          float64
 1   High          float64
 2   Low           float64
 3   Close         float64
 4   Volume        float64
 5   NumOfTrades   float64
 6   OHLCAvg       float64
 7   HLCAvg        float64
 8   HLAvg         float64
 9   BidVolume     float64
 10  AskVolume     float64
 11  fsvwap        float64
 12  dPOC          float64
 13  dVAH          float64
 14  dVAL          float64
 15  rthvwap       float64
 16  onPOC         float64
 17  onVAH         float64
 18  onVAL         float64
 19  dHi           float64
 20  dLo           float64
 21  dMid          float64
 22  IBH5          float64
 23  IBL5          float64
 24  IBH15         float64
 25  IBL15         float64
 26  IBH30         float64
 27  IBL30         float64
 28  IBH           float64
 29  IBL         

In [32]:
# data_es to parquet
data_es.to_parquet('data_es.parquet', engine='pyarrow', compression='snappy')

In [33]:
# data_nq to parquet
data_nq.to_parquet('data_nq.parquet', engine='pyarrow', compression='snappy')

In [34]:
# let's check the parquet files
data_es = pd.read_parquet('data_es.parquet', engine='pyarrow')
data_nq = pd.read_parquet('data_nq.parquet', engine='pyarrow')

In [35]:
# check the data
data_es.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2703600 entries, 2018-08-02 08:30:00 to 2024-08-08 15:59:45
Data columns (total 39 columns):
 #   Column        Dtype  
---  ------        -----  
 0   Open          float64
 1   High          float64
 2   Low           float64
 3   Close         float64
 4   Volume        float64
 5   NumOfTrades   float64
 6   OHLCAvg       float64
 7   HLCAvg        float64
 8   HLAvg         float64
 9   BidVolume     float64
 10  AskVolume     float64
 11  fsvwap        float64
 12  dPOC          float64
 13  dVAH          float64
 14  dVAL          float64
 15  rthvwap       float64
 16  onPOC         float64
 17  onVAH         float64
 18  onVAL         float64
 19  dHi           float64
 20  dLo           float64
 21  dMid          float64
 22  IBH5          float64
 23  IBL5          float64
 24  IBH15         float64
 25  IBL15         float64
 26  IBH30         float64
 27  IBL30         float64
 28  IBH           float64
 29  IBL         

In [36]:
data_es.head()

,Open,High,Low,Close,Volume,NumOfTrades,OHLCAvg,HLCAvg,HLAvg,BidVolume,AskVolume,fsvwap,dPOC,dVAH,dVAL,rthvwap,onPOC,onVAH,onVAL,dHi,dLo,dMid,IBH5,IBL5,IBH15,IBL15,IBH30,IBL30,IBH,IBL,ONH,ONL,ONMID,pHi,pLo,pMid,pCl,score,AvgTradeSize
2018-08-02 08:30:00,3095.50,3095.75,3093.25,3093.75,5183.0,2017.0,3094.56,3094.25,3094.50,3042.0,2141.0,3098.1,3093.75,3095.00,3093.50,3094.39,3093.25,3100.75,3089.5,3095.75,3093.25,3094.500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3113.25,3089.25,3101.25,NaN,NaN,NaN,NaN,0.0,2.57
2018-08-02 08:30:15,3093.50,3094.75,3093.50,3094.75,2644.0,971.0,3094.13,3094.33,3094.13,1265.0,1379.0,3098.1,3094.50,3094.75,3093.50,3094.36,3093.25,3100.75,3089.5,3095.75,3093.25,3094.500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3113.25,3089.25,3101.25,NaN,NaN,NaN,NaN,0.0,2.72
2018-08-02 08:30:30,3094.75,3095.25,3094.50,3094.50,1298.0,547.0,3094.75,3094.75,3094.88,862.0,436.0,3098.1,3094.50,3095.00,3093.75,3094.43,3093.25,3100.75,3089.5,3095.75,3093.25,3094.500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3113.25,3089.25,3101.25,NaN,NaN,NaN,NaN,25.0,2.37
2018-08-02 08:30:45,3094.25,3095.00,3093.75,3094.25,2162.0,757.0,3094.31,3094.33,3094.38,1143.0,1019.0,3098.0,3094.50,3094.75,3093.75,3094.43,3093.25,3100.75,3089.5,3095.75,3093.25,3094.500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3113.25,3089.25,3101.25,NaN,NaN,NaN,NaN,5.0,2.86
2018-08-02 08:31:00,3094.25,3096.00,3094.00,3095.75,1970.0,840.0,3095.00,3095.25,3095.00,947.0,1023.0,3098.0,3094.25,3095.00,3093.75,3094.55,3093.25,3100.75,3089.5,3096.00,3093.25,3094.625,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3113.25,3089.25,3101.25,NaN,NaN,NaN,NaN,5.0,2.35


In [37]:
data_es.tail()

,Open,High,Low,Close,Volume,NumOfTrades,OHLCAvg,HLCAvg,HLAvg,BidVolume,AskVolume,fsvwap,dPOC,dVAH,dVAL,rthvwap,onPOC,onVAH,onVAL,dHi,dLo,dMid,IBH5,IBL5,IBH15,IBL15,IBH30,IBL30,IBH,IBL,ONH,ONL,ONMID,pHi,pLo,pMid,pCl,score,AvgTradeSize
2024-08-08 15:58:45,5353.75,5353.75,5353.25,5353.75,130.0,79.0,5353.63,5353.58,5353.50,66.0,64.0,5303.2,5344.25,5356.5,5311.75,5321.53,5231.5,5241.75,5182.0,5361.0,5259.75,5310.375,5285.0,5272.75,5285.5,5271.0,5292.5,5259.75,5322.75,5259.75,5285.25,5182.0,5233.625,5359.25,5196.75,5278.0,5202.25,80.0,1.65
2024-08-08 15:59:00,5353.75,5353.75,5353.00,5353.25,112.0,83.0,5353.44,5353.33,5353.38,62.0,50.0,5303.2,5344.25,5356.5,5311.75,5321.53,5231.5,5241.75,5182.0,5361.0,5259.75,5310.375,5285.0,5272.75,5285.5,5271.0,5292.5,5259.75,5322.75,5259.75,5285.25,5182.0,5233.625,5359.25,5196.75,5278.0,5202.25,80.0,1.35
2024-08-08 15:59:15,5353.25,5353.50,5353.00,5353.50,56.0,40.0,5353.31,5353.33,5353.25,28.0,28.0,5303.2,5344.25,5356.5,5311.75,5321.53,5231.5,5241.75,5182.0,5361.0,5259.75,5310.375,5285.0,5272.75,5285.5,5271.0,5292.5,5259.75,5322.75,5259.75,5285.25,5182.0,5233.625,5359.25,5196.75,5278.0,5202.25,80.0,1.40
2024-08-08 15:59:30,5353.25,5353.50,5353.00,5353.00,71.0,55.0,5353.19,5353.17,5353.25,50.0,21.0,5303.2,5344.25,5356.5,5311.75,5321.53,5231.5,5241.75,5182.0,5361.0,5259.75,5310.375,5285.0,5272.75,5285.5,5271.0,5292.5,5259.75,5322.75,5259.75,5285.25,5182.0,5233.625,5359.25,5196.75,5278.0,5202.25,80.0,1.29
2024-08-08 15:59:45,5353.00,5353.50,5350.50,5353.00,339.0,238.0,5352.50,5352.33,5352.00,198.0,141.0,5303.2,5344.25,5356.5,5311.75,5321.54,5231.5,5241.75,5182.0,5361.0,5259.75,5310.375,5285.0,5272.75,5285.5,5271.0,5292.5,5259.75,5322.75,5259.75,5285.25,5182.0,5233.625,5359.25,5196.75,5278.0,5202.25,80.0,1.42
